In [ ]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd
import statistics
import random
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.models import model_from_json
from sklearn.metrics import log_loss
import scipy.misc
# from scipy.misc import imread, imresize


In [ ]:
import pickle
import datetime
import pandas as pd
import keras

from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, \
                                       ZeroPadding2D

# from keras.layers.normalization import BatchNormalization
# from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import model_from_json
# from sklearn.metrics import log_loss
from numpy.random import permutation
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/deep_learning_project/

/content/drive/.shortcut-targets-by-id/15gKjVs5Brz2wVAZdIJoqScr7ejFRXBo9/deep_learning_project


In [ ]:
#Load Data
train_data = np.load('train_data.npy')
train_target = np.load('train_target.npy')
driver_id = np.load('driver_id.npy')
unique_drivers = np.load('unique_drivers.npy')

# Simple CNN

In [ ]:

def create_model_v1(img_rows, img_cols, color_type=1):
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, padding='same', kernel_initializer='he_normal',
                            input_shape=(color_type, img_rows, img_cols)))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(64, 3, 3, padding='same', kernel_initializer='he_normal'))
    model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(128, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(MaxPooling2D((8, 8),padding='same'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(Adam(lr=0.001), loss='categorical_crossentropy',metrics=['accuracy'])
    return model
    
def run_single():
    # input image dimensions
    img_rows, img_cols = 64, 64
    batch_size = 32
    nb_epoch = 100


    yfull_train = dict()
    yfull_test = []

    unique_list_train = ['p002', 'p012', 'p014', 'p016', 'p021', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p051', 'p052', 'p061', 'p064', 'p066', 'p072',
                     'p075','p081']
    X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
    unique_list_valid = ['p050','p015','p022','p056']
    X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

    print('Start Single Run')
    print('Split train: ', len(X_train), len(Y_train))
    print('Split valid: ', len(X_valid), len(Y_valid))
    print('Train drivers: ', unique_list_train)
    print('Test drivers: ', unique_list_valid)
    
    model = create_model_v1(img_rows, img_cols, color_type_global)
    callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)
    his = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch
                    ,verbose=1, validation_data=(X_valid, Y_valid),callbacks=[callback])
    
    return his




his = run_single()

Start Single Run
Split train:  18732 18732
Split valid:  3692 3692
Train drivers:  ['p002', 'p012', 'p014', 'p016', 'p021', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p051', 'p052', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']
Test drivers:  ['p050', 'p015', 'p022', 'p056']
Epoch 1/100
586/586 [==============================] - 4s 5ms/step - loss: 2.3042 - accuracy: 0.1363 - val_loss: 2.2639 - val_accuracy: 0.1631
Epoch 2/100
586/586 [==============================] - 3s 5ms/step - loss: 1.9272 - accuracy: 0.2989 - val_loss: 2.3623 - val_accuracy: 0.1896
Epoch 3/100
586/586 [==============================] - 3s 5ms/step - loss: 1.7376 - accuracy: 0.3669 - val_loss: 2.3039 - val_accuracy: 0.2511
Epoch 4/100
586/586 [==============================] - 3s 5ms/step - loss: 1.6173 - accuracy: 0.4187 - val_loss: 2.4949 - val_accuracy: 0.1953
Epoch 5/100
586/586 [==============================] - 3s 5ms/step - loss: 1.5517 - accuracy: 0.4394 - val_loss: 2.3556 

In [ ]:
#Save result
create_model_v1 = pd.DataFrame(his.history)
create_model_v1.to_csv('create_model_v1.csv')

# VGG_BN

In [ ]:
def vgg_bn(img_rows, img_cols, color_type=1):
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), padding='same', kernel_initializer='he_normal', input_shape=(color_type, img_rows, img_cols)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(32, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), padding='same',strides=(2, 2),kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(128, (3, 3), padding='same',strides=(2, 2),kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(128, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='sigmoid', kernel_initializer='he_normal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax', kernel_initializer='he_normal'))
    model.compile(Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
img_rows, img_cols = 64, 64
batch_size = 32
nb_epoch = 100

#Train test split
yfull_train = dict()
yfull_test = []
unique_list_train = ['p002', 'p012', 'p014', 'p016', 'p021', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p051', 'p052', 'p061', 'p064', 'p066', 'p072',
                     'p075','p081']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p050','p015','p022','p056']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)

#Run model
model = vgg_bn(img_rows, img_cols, color_type_global)
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)
hist = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch
                 ,verbose=1, validation_data=(X_valid, Y_valid),callbacks=[callback])

Start Single Run
Split train:  18732 18732
Split valid:  3692 3692
Train drivers:  ['p002', 'p012', 'p014', 'p016', 'p021', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p051', 'p052', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']
Test drivers:  ['p050', 'p015', 'p022', 'p056']
Epoch 1/100
586/586 [==============================] - 6s 8ms/step - loss: 1.6738 - accuracy: 0.4076 - val_loss: 2.6448 - val_accuracy: 0.2793
Epoch 2/100
586/586 [==============================] - 4s 7ms/step - loss: 0.4518 - accuracy: 0.8644 - val_loss: 2.5857 - val_accuracy: 0.3237
Epoch 3/100
586/586 [==============================] - 4s 7ms/step - loss: 0.2506 - accuracy: 0.9270 - val_loss: 2.7102 - val_accuracy: 0.3459
Epoch 4/100
586/586 [==============================] - 4s 7ms/step - loss: 0.1694 - accuracy: 0.9532 - val_loss: 2.9252 - val_accuracy: 0.3191
Epoch 5/100
586/586 [==============================] - 4s 7ms/step - loss: 0.1385 - accuracy: 0.9620 - val_loss: 3.7391 

In [ ]:
#Save result
vgg_bn = pd.DataFrame(hist.history)
vgg_bn.to_csv('vgg_bn.csv')

# Data augment before vgg_bn

In [ ]:
def vgg_bn(img_rows, img_cols, color_type=1):
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), padding='same', kernel_initializer='he_normal', input_shape=(color_type, img_rows, img_cols)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(32, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), padding='same',strides=(2, 2),kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(128, (3, 3), padding='same',strides=(2, 2),kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Convolution2D(128, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='sigmoid', kernel_initializer='he_normal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax', kernel_initializer='he_normal'))
    model.compile(Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping
import time

#Record time
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()
#Early stop
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)


#data augmentation
datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30
        )

data_generator = datagen.flow(X_train, Y_train, batch_size = 64)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (18732, 1, 64, 64) (64 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


In [ ]:
img_rows, img_cols = 64, 64
batch_size = 64
nb_epoch = 100

#Train test split
yfull_train = dict()
yfull_test = []
unique_list_train = ['p002', 'p012', 'p014', 'p016', 'p021', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p051', 'p052', 'p061', 'p064', 'p066', 'p072',
                     'p075','p081']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p050','p015','p022','p056']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)

#Run model
model = vgg_bn(img_rows, img_cols, 1)
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)
hist = model.fit_generator(data_generator,steps_per_epoch = len(X_train) / 64, epochs=nb_epoch
                 ,verbose=1, validation_data=(X_valid, Y_valid),callbacks=[callback,time_callback])

Start Single Run
Split train:  18732 18732
Split valid:  3692 3692
Train drivers:  ['p002', 'p012', 'p014', 'p016', 'p021', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p051', 'p052', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']
Test drivers:  ['p050', 'p015', 'p022', 'p056']


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
292/292 [==============================] - 51s 170ms/step - loss: 2.2719 - accuracy: 0.1614 - val_loss: 2.7994 - val_accuracy: 0.1446
Epoch 2/100
292/292 [==============================] - 49s 169ms/step - loss: 1.6517 - accuracy: 0.3775 - val_loss: 2.9988 - val_accuracy: 0.2183
Epoch 3/100
292/292 [==============================] - 49s 169ms/step - loss: 1.3976 - accuracy: 0.4722 - val_loss: 2.5813 - val_accuracy: 0.2224
Epoch 4/100
292/292 [==============================] - 49s 169ms/step - loss: 1.2109 - accuracy: 0.5611 - val_loss: 2.5914 - val_accuracy: 0.2470
Epoch 5/100
292/292 [==============================] - 49s 169ms/step - loss: 1.0543 - accuracy: 0.6341 - val_loss: 2.8665 - val_accuracy: 0.2557
Epoch 6/100
292/292 [==============================] - 50s 169ms/step - loss: 0.9511 - accuracy: 0.6716 - val_loss: 2.7273 - val_accuracy: 0.2589
Epoch 7/100
292/292 [==============================] - 50s 169ms/step - loss: 0.8563 - accuracy: 0.7135 - val_loss: 2.9760 -

# Another simple CNN struture

In [ ]:
def cnn(img_rows, img_cols, color_type=1):
  model = Sequential()

  model.add(Convolution2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(color_type, img_rows, img_cols), kernel_initializer='glorot_normal'))
  model.add(MaxPooling2D(pool_size=2,padding='same'))
  model.add(Convolution2D(filters=128, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
  model.add(MaxPooling2D(pool_size=2,padding='same'))
  model.add(Convolution2D(filters=256, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
  model.add(MaxPooling2D(pool_size=2,padding='same'))
  model.add(Convolution2D(filters=512, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
  model.add(MaxPooling2D(pool_size=2,padding='same'))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(500, activation='relu', kernel_initializer='glorot_normal'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax', kernel_initializer='glorot_normal'))
  model.compile(Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
from sklearn.model_selection import train_test_split
img_rows, img_cols = 64, 64
batch_size = 32
nb_epoch = 100



unique_list_train = ['p002', 'p012', 'p014', 'p016', 'p021', 'p024',
                     'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049',
                     'p051', 'p052', 'p061', 'p064', 'p066', 'p072',
                     'p075','p081']
X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
unique_list_valid = ['p050','p015','p022','p056']
X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

print('Start Single Run')
print('Split train: ', len(X_train), len(Y_train))
print('Split valid: ', len(X_valid), len(Y_valid))
print('Train drivers: ', unique_list_train)
print('Test drivers: ', unique_list_valid)

model = cnn(img_rows, img_cols, color_type_global)
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)
his = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch
                ,verbose=1, validation_data=(X_valid, Y_valid),callbacks=[callback])

Start Single Run
Split train:  18732 18732
Split valid:  3692 3692
Train drivers:  ['p002', 'p012', 'p014', 'p016', 'p021', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p051', 'p052', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']
Test drivers:  ['p050', 'p015', 'p022', 'p056']
Epoch 1/100
586/586 [==============================] - 5s 7ms/step - loss: 1.9273 - accuracy: 0.2667 - val_loss: 2.7315 - val_accuracy: 0.2397
Epoch 2/100
586/586 [==============================] - 4s 6ms/step - loss: 1.0399 - accuracy: 0.6185 - val_loss: 3.0620 - val_accuracy: 0.2638
Epoch 3/100
586/586 [==============================] - 4s 6ms/step - loss: 0.5880 - accuracy: 0.7959 - val_loss: 3.2944 - val_accuracy: 0.3150
Epoch 4/100
586/586 [==============================] - 4s 6ms/step - loss: 0.3826 - accuracy: 0.8788 - val_loss: 3.2624 - val_accuracy: 0.3402
Epoch 5/100
586/586 [==============================] - 4s 6ms/step - loss: 0.2807 - accuracy: 0.9095 - val_loss: 3.3131 

In [ ]:
cnn = pd.DataFrame(his.history)
cnn.to_csv('cnn.csv')

# ResNet50

In [ ]:
X_train=np.load('/content/drive/MyDrive/deep_learning_project/X_train.npy')
X_test=np.load('/content/drive/MyDrive/deep_learning_project/X_test.npy')
y_train=np.load('/content/drive/MyDrive/deep_learning_project/y_train.npy')
y_test=np.load('/content/drive/MyDrive/deep_learning_project/y_test.npy')

In [ ]:
import tensorflow.keras as K
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf

model_input_shape = (224,224,3)
base_model  =K.applications.resnet.ResNet50(include_top = False,
                                                  weights = 'imagenet',
                                                  input_shape = model_input_shape)
base_model.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________

In [ ]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

time_callback = TimeHistory()
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20)

x = base_model.output
x = K.layers.Flatten()(x)
x = K.layers.Dropout(0.5)(x)

output =K.layers.Dense(units = 10,activation = tf.nn.softmax)(x)
model = K.models.Model(inputs=base_model.inputs, outputs=output)

model.compile(optimizer=K.optimizers.Adam(0.0001),
              loss=K.losses.CategoricalCrossentropy(from_logits = False),
              metrics=['accuracy'])

In [ ]:
hist = model.fit(
    x = X_train,y=y_train,
    validation_data=(X_test,y_test),
    batch_size = 64,
    epochs=100,
    callbacks = [callback,time_callback],
    verbose=1)

Epoch 1/100
270/270 [==============================] - 58s 198ms/step - loss: 0.7466 - accuracy: 0.8483 - val_loss: 1.0245 - val_accuracy: 0.7701
Epoch 2/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0140 - accuracy: 0.9955 - val_loss: 0.8076 - val_accuracy: 0.8080
Epoch 3/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0053 - accuracy: 0.9988 - val_loss: 0.7510 - val_accuracy: 0.8235
Epoch 4/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0121 - accuracy: 0.9967 - val_loss: 1.7037 - val_accuracy: 0.7748
Epoch 5/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0278 - accuracy: 0.9941 - val_loss: 0.7766 - val_accuracy: 0.8523
Epoch 6/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0172 - accuracy: 0.9967 - val_loss: 0.6524 - val_accuracy: 0.8570
Epoch 7/100
270/270 [==============================] - 53s 195ms/step - loss: 0.0058 - accuracy: 0.9983 - val_loss: 0.9559 -

In [ ]:
res_50_no_augment = pd.DataFrame(hist.history)
res_50_no_augment['time'] = time_callback.times
res_50_no_augment.to_csv('res_50_no_augment.csv')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping
import time



#data augmentation
datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30
        )
#datagen.fit(X_train)
data_generator = datagen.flow(X_train, y_train, batch_size = 64)

# Fits the model on batches with real-time data augmentation:
hist = model.fit_generator(data_generator,steps_per_epoch = len(X_train) / 64, callbacks=[time_callback],
                                                   epochs = 50, verbose = 1, validation_data = (X_test, y_test))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
269/269 [==============================] - 215s 783ms/step - loss: 1.5817 - accuracy: 0.5884 - val_loss: 0.5572 - val_accuracy: 0.8664
Epoch 2/50
269/269 [==============================] - 210s 777ms/step - loss: 0.7090 - accuracy: 0.7684 - val_loss: 1.0661 - val_accuracy: 0.7563
Epoch 3/50
269/269 [==============================] - 215s 795ms/step - loss: 0.5608 - accuracy: 0.8183 - val_loss: 0.9638 - val_accuracy: 0.7707
Epoch 4/50
269/269 [==============================] - 215s 795ms/step - loss: 0.4683 - accuracy: 0.8413 - val_loss: 0.5267 - val_accuracy: 0.8808
Epoch 5/50
269/269 [==============================] - 214s 793ms/step - loss: 0.4129 - accuracy: 0.8596 - val_loss: 1.3406 - val_accuracy: 0.7739
Epoch 6/50
269/269 [==============================] - 213s 790ms/step - loss: 0.4200 - accuracy: 0.8618 - val_loss: 0.8258 - val_accuracy: 0.8602
Epoch 7/50
269/269 [==============================] - 214s 793ms/step - loss: 0.3485 - accuracy: 0.8825 - val_loss: 0.4853 -

In [ ]:
res_50_with_augment = pd.DataFrame(hist.history)
res_50_with_augment['time'] = time_callback.times
res_50_with_augment.to_csv('res_50_with_augment.csv')